In [1]:
# ======================================================================================
# Notebook setup
# 
# Run this cell before all others to make sure that the Jupyter notebook works properly
# ======================================================================================

# Automatically reload all imported modules
%load_ext autoreload
%autoreload 2

figsize = (14, 3.5)

# Exercise: Seoul Bike Sharing

## Loading and Preprocessing as Usual

**We will try using ensemble models on the Seoul Bike Sharing dataset**

First, let's load and preprocess the data as usual (for tree-based models)

In [2]:
import pandas as pd
import os

fname = os.path.join('data', 'SeoulBikeData.csv')
data = pd.read_csv(fname, sep=',')

num_input = ['Temperature(^C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(^C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)']
cat_input = ['Hour', 'Seasons', 'Holiday', 'Functioning Day']
output = 'Rented Bike Count'

data_flt = data[num_input + cat_input + [output]].copy()
for cname in cat_input:
    data_flt[cname] = data_flt[cname].astype('category').cat.codes

data_flt.head()

,Temperature(^C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(^C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Hour,Seasons,Holiday,Functioning Day,Rented Bike Count
0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,0,3,1,1,254
1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,1,3,1,1,204
2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,2,3,1,1,173
3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,3,3,1,1,107
4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,4,3,1,1,78


In [3]:
from sklearn.model_selection import train_test_split

X = data_flt[num_input + cat_input]
y = data_flt[output] # We use a list, so we obtain a DataFrame

test_set_fraction = .34
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=test_set_fraction, random_state=42)

print(f"Number of training examples: {len(X_tr)}")
print(f"Number of test examples: {len(X_ts)}")

Number of training examples: 5781
Number of test examples: 2979


## Learning Ensemble Models

* Try learnign a Random Forest for this problem
* If possible, try also calibrating the parameters using grid search and cross-validation

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'n_estimators': np.arange(150, 260, 50), 'max_depth': np.arange(10, 14, 2)}
gcv = GridSearchCV(RandomForestRegressor(), param_grid=param_grid)

gcv.fit(X_tr, y_tr)

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': array([10, 12]),
                         'n_estimators': array([150, 200, 250])})

In [17]:
from sklearn.metrics import r2_score

gcv_pred_tr, gcv_pred_ts = gcv.predict(X_tr), gcv.predict(X_ts)
print(f'R2: {r2_score(y_tr, gcv_pred_tr):.3} (training), {r2_score(y_ts, gcv_pred_ts):.3} (test)')
print(f'Best results with: {gcv.best_params_}')

R2: 0.949 (training), 0.859 (test)
Best results with: {'max_depth': 12, 'n_estimators': 250}


* Then, try learnign a Gradient Boosted Tree model for this problem
* If possible, try also calibrating the parameters using grid search and cross-validation

In [19]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid2 = {'n_estimators': np.arange(200, 260, 50), 'max_depth': np.arange(2, 5, 1)}
gcv2 = GridSearchCV(GradientBoostingRegressor(), param_grid=param_grid2)

gcv2.fit(X_tr, y_tr)

GridSearchCV(estimator=GradientBoostingRegressor(),
             param_grid={'max_depth': array([2, 3, 4]),
                         'n_estimators': array([200, 250])})

In [20]:
gcv2_pred_tr, gcv2_pred_ts = gcv2.predict(X_tr), gcv2.predict(X_ts)
print(f'R2: {r2_score(y_tr, gcv2_pred_tr):.3} (training), {r2_score(y_ts, gcv2_pred_ts):.3} (test)')
print(f'Best results with: {gcv2.best_params_}')

R2: 0.92 (training), 0.862 (test)
Best results with: {'max_depth': 4, 'n_estimators': 250}
